# CEE 697M - Final Project: Jerry Chen and Aniruddha Prasad


In [2]:
!pip install pyreadstat

     |████████████████████████████████| 2.4 MB 4.6 MB/s 


## Import Required Libraries:

In [10]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns


## Import the data:

In [11]:
df = pd.read_spss("sample_data/1ResearchProjectData.sav")

## Pre-process the data

In [12]:
df

,Student,Teacher,Gender,Ethnic,Freeredu,Score,wesson
0,1.0,Ruger,Female,Asian,Free lunch,76.0,Ruger_Smith
1,2.0,Ruger,Female,Hispanic,Paid lunch,56.0,Ruger_Smith
2,3.0,Ruger,Female,African-American,Free lunch,34.0,Ruger_Smith
3,4.0,Ruger,Female,Asian,Paid lunch,59.0,Ruger_Smith
4,5.0,Ruger,Male,Hispanic,Free lunch,73.0,Ruger_Smith
...,...,...,...,...,...,...,...
212,213.0,Wesson,Male,Hispanic,Free lunch,94.0,Wesson
213,214.0,Wesson,Male,Hispanic,Paid lunch,91.0,Wesson
214,215.0,Wesson,Female,African-American,Paid lunch,53.0,Wesson
215,216.0,Wesson,Male,Hispanic,Paid lunch,57.0,Wesson


## Split data into Training and Validation sets: